In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role
import os

# Oturum aç
sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = sagemaker_session.boto_region_name
default_bucket = sagemaker_session.default_bucket()

# --- YENİ BAŞLANGIÇ: Versiyon 2 ---
# Eski dosyalarla karışmasın diye v2 klasörü kullanıyoruz
project_prefix = "tez-v2"

print(f"✅ Sistem Hazır. Çalışma Alanı: s3://{default_bucket}/{project_prefix}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
✅ Sistem Hazır. Çalışma Alanı: s3://sagemaker-us-east-1-437151405779/tez-v2


In [3]:
local_path = "adult.csv"
s3_input_uri = f"s3://{default_bucket}/{project_prefix}/data/adult.csv"

if os.path.exists(local_path):
    print(f"Dosya bulundu, S3'e yükleniyor: {s3_input_uri}")
    sagemaker.s3.S3Uploader.upload(local_path, s3_input_uri)
    print("✅ Veri Yüklendi.")
else:
    print("❌ HATA: adult.csv dosyasını sol tarafa yüklememişsin!")

Dosya bulundu, S3'e yükleniyor: s3://sagemaker-us-east-1-437151405779/tez-v2/data/adult.csv
✅ Veri Yüklendi.


In [4]:
%%writefile preprocessing.py
import argparse
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

if __name__ == "__main__":
    print("Veri isleme basliyor...")
    base_dir = "/opt/ml/processing"
    
    # Klasörleri oluştur
    os.makedirs(f"{base_dir}/train", exist_ok=True)
    os.makedirs(f"{base_dir}/test", exist_ok=True)
    
    # Veriyi oku (Boşlukları temizleyerek)
    df = pd.read_csv(f"{base_dir}/input/adult.csv", header=None, skipinitialspace=True)
    
    # Sütun isimleri
    columns = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
               "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
               "hours-per-week", "native-country", "income"]
    df.columns = columns

    # Hedef değişkeni temizle
    df['income'] = df['income'].astype(str).str.strip().apply(lambda x: 1 if '>50K' in x else 0)
    
    # Sadece sayısal sütunları al
    df = df.replace('?', np.nan).dropna()
    df = df.select_dtypes(include=['int64', 'float64'])
    
    # Train/Test ayır
    train, test = train_test_split(df, test_size=0.2, random_state=42)
    
    # Kaydet
    train.to_csv(f"{base_dir}/train/train.csv", header=False, index=False)
    test.to_csv(f"{base_dir}/test/test.csv", header=False, index=False)
    print(f"✅ Tamamlandı. Train: {len(train)}, Test: {len(test)}")

Writing preprocessing.py


In [5]:
%%writefile train.py
import argparse
import os
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score

if __name__ == "__main__":
    print("Egitim basliyor...")
    
    # Okuma
    train_data = pd.read_csv("/opt/ml/input/data/train/train.csv", header=None)
    val_data = pd.read_csv("/opt/ml/input/data/validation/test.csv", header=None)
    
    X_train = train_data.iloc[:, :-1]
    y_train = train_data.iloc[:, -1]
    X_val = val_data.iloc[:, :-1]
    y_val = val_data.iloc[:, -1]
    
    # Eğitim
    model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='error')
    model.fit(X_train, y_train)
    
    # Kayıt (XGBoost native format - Hata vermez)
    model.get_booster().save_model("/opt/ml/model/xgboost-model")
    print("✅ Model kaydedildi.")

Writing train.py


In [6]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep, TrainingStep
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.pipeline import Pipeline

# --- SABİT ADRES STRATEJİSİ ---
# İşlenen veriler ve modeller tam olarak buraya gidecek.
processed_data_uri = f"s3://{default_bucket}/{project_prefix}/processed"
model_output_uri = f"s3://{default_bucket}/{project_prefix}/models"

# 1. İşlemci Ayarları
sklearn_processor = SKLearnProcessor(
    framework_version="0.23-1",
    role=role,
    instance_type="ml.t3.medium",
    instance_count=1,
    base_job_name="clean-process"
)

# 2. Veri İşleme Adımı (Çıktı adresini elle veriyoruz)
step_process = ProcessingStep(
    name="VeriIsleme_v2",
    processor=sklearn_processor,
    inputs=[ProcessingInput(source=s3_input_uri, destination="/opt/ml/processing/input")],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train", destination=f"{processed_data_uri}/train"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test", destination=f"{processed_data_uri}/test")
    ],
    code="preprocessing.py"
)

# 3. Eğitim Ayarları
xgb_estimator = Estimator(
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    image_uri=sagemaker.image_uris.retrieve("xgboost", region, "1.5-1"),
    output_path=model_output_uri,
    entry_point="train.py",
    source_dir="."
)

# 4. Eğitim Adımı (Girdi adresini elle veriyoruz)
step_train = TrainingStep(
    name="ModelEgitimi_v2",
    estimator=xgb_estimator,
    inputs={
        "train": TrainingInput(s3_data=f"{processed_data_uri}/train", content_type="text/csv"),
        "validation": TrainingInput(s3_data=f"{processed_data_uri}/test", content_type="text/csv")
    },
    depends_on=[step_process] # İşleme bitmeden başlama
)

# 5. Başlat
pipeline = Pipeline(
    name="Tez-Pipeline-v2",
    steps=[step_process, step_train],
    sagemaker_session=sagemaker_session
)

pipeline.upsert(role_arn=role)
execution = pipeline.start()
print(f"🚀 Pipeline v2 Fırlatıldı! ARN: {execution.arn}")

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


🚀 Pipeline v2 Fırlatıldı! ARN: arn:aws:sagemaker:us-east-1:437151405779:pipeline/Tez-Pipeline-v2/execution/5qsubr80h8or


In [7]:
# Son çalışan adımları listele
steps = execution.list_steps()

# Eğitim adımını bul
training_step = next(s for s in steps if s['StepName'] == 'ModelEgitimi_v2')

# Eğitim işinin detaylarına inip modelin S3 adresini alıyoruz
training_job_arn = training_step['Metadata']['TrainingJob']['Arn']
training_job_name = training_job_arn.split('/')[-1]
desc = sagemaker_session.sagemaker_client.describe_training_job(TrainingJobName=training_job_name)
model_s3_uri = desc['ModelArtifacts']['S3ModelArtifacts']

print(f"✅ Model Adresi Bulundu: {model_s3_uri}")

# Modelin çalışacağı imajı (XGBoost) tekrar çağırıyoruz
image_uri = sagemaker.image_uris.retrieve("xgboost", region, "1.5-1")

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


✅ Model Adresi Bulundu: s3://sagemaker-us-east-1-437151405779/tez-v2/models/pipelines-5qsubr80h8or-ModelEgitimi-v2-iigI3wBPIL/output/model.tar.gz


In [8]:
from sagemaker.model import Model

# Model nesnesini hazırlıyoruz
model = Model(
    image_uri=image_uri,
    model_data=model_s3_uri,
    role=role,
    sagemaker_session=sagemaker_session
)

print(f"🚀 Endpoint kuruluyor... (Bu işlem 5-10 dk sürebilir, kahveni tazele ☕)")

# Deploy ediyoruz (Free tier dostu makineyle)
predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.t2.medium"
)

print(f"\n✅ Endpoint Başarıyla Kuruldu! Adı: {predictor.endpoint_name}")

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2025-12-01-12-41-18-432


🚀 Endpoint kuruluyor... (Bu işlem 5-10 dk sürebilir, kahveni tazele ☕)


INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2025-12-01-12-41-19-123
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2025-12-01-12-41-19-123


------------!

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:19                                                                                   │
│                                                                                                  │
│   16 │   instance_type="ml.t2.medium"                                                            │
│   17 )                                                                                           │
│   18                                                                                             │
│ ❱ 19 print(f"\n✅ Endpoint Başarıyla Kuruldu! Adı: {predictor.endpoint_name}")                   │
│   20                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'NoneType' object has no attribute 'endpoint_name'

In [9]:
import boto3
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer

# 1. AWS'ye sor: "Çalışan en yeni endpoint hangisi?"
sm_client = boto3.client("sagemaker")
endpoints = sm_client.list_endpoints(SortBy='CreationTime', SortOrder='Descending', StatusEquals='InService')

if len(endpoints['Endpoints']) > 0:
    # İsmi otomatik bulduk
    endpoint_ismi = endpoints['Endpoints'][0]['EndpointName']
    print(f"✅ BULUNDU! Aktif Endpoint: {endpoint_ismi}")
    
    # 2. Bağlanıyoruz (Kumandayı geri alıyoruz)
    predictor = Predictor(
        endpoint_name=endpoint_ismi,
        sagemaker_session=sagemaker_session
    )
    predictor.serializer = CSVSerializer()
    
    print("🚀 Bağlantı kuruldu! Şimdi Hücre 9 ile test edebilirsin.")
else:
    print("❌ HATA: Şu an 'InService' durumunda endpoint bulunamadı. Biraz daha beklemen gerekebilir.")

✅ BULUNDU! Aktif Endpoint: sagemaker-xgboost-2025-12-01-12-41-19-123
🚀 Bağlantı kuruldu! Şimdi Hücre 9 ile test edebilirsin.


In [10]:
from sagemaker.serializers import CSVSerializer

# Sunucuyla CSV diliyle konuşacağımızı belirtiyoruz
predictor.serializer = CSVSerializer()

# TEST VERİSİ:
# 39 Yaşında, 77K gelirli, 13 yıl okumuş, sermaye kazancı 2174 dolar vb.
test_verisi = "39, 77516, 13, 2174, 0, 40" 

print(f"❓ Soru Soruluyor: {test_verisi}")
tahmin = predictor.predict(test_verisi)
sonuc = float(tahmin.decode('utf-8'))

print(f"🤖 Modelin Skoru: {sonuc}")

if sonuc > 0.5:
    print("💰 TAHMİN: >50K (Zengin)")
else:
    print("📉 TAHMİN: <=50K (Fakir)")

❓ Soru Soruluyor: 39, 77516, 13, 2174, 0, 40
🤖 Modelin Skoru: 0.004923452157527208
📉 TAHMİN: <=50K (Fakir)


In [11]:
 predictor.delete_endpoint()
 print("✅ Endpoint silindi. Görev tamamlandı Tony! 🏁")

INFO:sagemaker:Deleting endpoint configuration with name: sagemaker-xgboost-2025-12-01-12-41-19-123
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2025-12-01-12-41-19-123


✅ Endpoint silindi. Görev tamamlandı Tony! 🏁
